In [3]:
import numpy as np
from tool.utils import *
from models import *
import cv2
import torch
from torch import nn
import torch.nn.functional as F
import copy

In [4]:
model = Yolov4(n_classes=2)
pretrained_dict = torch.load('checkpoints/Yolov4_epoch300.pth', map_location=torch.device('cuda'))
model.load_state_dict(pretrained_dict)
model.cuda()

Yolov4(
  (down1): DownSample1(
    (conv1): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Mish()
      )
    )
    (conv2): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Mish()
      )
    )
    (conv3): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Mish()
      )
    )
    (conv4): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNo

In [ ]:
#conf_thres
for conf_thres in range(1,10,1):
    for nms_thres in range(1,10,1):
        truth = {}
        tp = np.zeros([2])
        fp = np.zeros([2])
        fn = np.zeros([2])
        tn = np.zeros([2])
        f = open('data/val.txt', 'r', encoding='utf-8')
        for line in f.readlines():
            data = line.split(" ")
            truth[data[0]] = []
            for i in data[1:]:
                truth[data[0]].append([int(j) for j in i.split(',')])
            #print(data[0])
            gt=truth[data[0]]
    
            img = cv2.imread('valid/'+data[0])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            h, w, _ = img.shape
            #print(h, w)
            sized = cv2.resize(img, (608, 608))
            boxes = do_detect(model, sized, conf_thres/10, 2, nms_thres/10, 1)
    
            # 模型偶爾沒辦法預測出bbox，塞入一個整張都是背景的bbox
            if len(boxes) == 0:
                boxes = np.array([[0.5, 0.5, 1, 1, 1.0, 0.0, 0.0]])
            gt_normal=copy.deepcopy(gt)
            #print(type(gt))
            for count,val in enumerate(gt): #normalize
                gt_normal[count][0]=((val[2]+val[0])/2.0)/w
                gt_normal[count][1]=((val[3]+val[1])/2.0)/h
                gt_normal[count][2]=(val[2]-val[0])/w
                gt_normal[count][3]=(val[3]-val[1])/h
        
            #print(gt_normal)
            gt_normal=np.array(gt_normal)
            #print(gt_normal)
            gt_normal=torch.from_numpy(gt_normal)
            #print(gt_normal)
            #gt_normal=torch.tensor(gt_normal)
            boxes=torch.tensor(boxes)
            #print(gt)
            #ious=np.zeros((len(gt_normal),len(boxes)))
            ious=np.zeros((len(boxes),len(gt_normal)))
            #print(ious)
    
            for cn1,i in enumerate(boxes): #ious
                for cn2,j in enumerate(gt_normal):
                    #print(ious.shape)
                    #print(cn2)
                    ious[cn1][cn2]=bbox_ious(i, j, x1y1x2y2=0)
            
            #開始算正確率
            #tp,fp,fn,tn=0,0,0,0
            #print('boxes')
            #print(boxes.size())
            boxes_cls=boxes[:,4:].numpy()
            boxes_cls_max=np.zeros([boxes.shape[0]], dtype=int)
            #print(boxes_cls_max.shape)
            for num,i in enumerate(boxes_cls):
                max_=0.0
                cn=0
                for index,j in enumerate(i):
                    if j>max_:
                        max_=j
                        cn=index
                boxes_cls_max[num]=cn
            #print(boxes_cls_max)
            #print(boxes_cls)
            for cn,i in enumerate(ious):
                count=0
                max_iou=0.0
                for index,j in enumerate(i) :
                    if j>max_iou:
                        count=index
                        max_iou=j
                if max_iou>=0.5:#若iou大於設定值，則參與計算正確率的計數器(TP、FP、FN、TN)中
                    #print('check1')
                    #Wear
                    if (gt_normal[count][4] == 0)&(boxes_cls_max[cn] == 1):
                        tp[0]+=1
                    if (gt_normal[count][4] != 0)&(boxes_cls_max[cn] == 1):
                        fp[0]+=1
                    if (gt_normal[count][4] == 0)&(boxes_cls_max[cn] != 1):
                        fn[0]+=1
                    if (gt_normal[count][4] != 0)&(boxes_cls_max[cn] != 1):
                        tn[0]+=1
                    #NotWear
                    if (gt_normal[count][4] == 1)&(boxes_cls_max[cn] == 2):
                        tp[1]+=1
                    if (gt_normal[count][4] != 1)&(boxes_cls_max[cn] == 2):
                        fp[1]+=1
                    if (gt_normal[count][4] == 1)&(boxes_cls_max[cn] != 2):
                        fn[1]+=1
                    if (gt_normal[count][4] != 1)&(boxes_cls_max[cn] != 2):
                        tn[1]+=1


        #print(tp,tn,fp,fn)
        acc = ((tp + tn)) / ((tp + tn + fp + fn))
        recall = tp / (tp+fn)
        prec = tp / (tp+fp)
        print('with conf_thresh:{} nms_thresh:{} acc={} recall={} prec={}'.format(conf_thres/10,nms_thres/10,acc,recall,prec))